<a href="https://colab.research.google.com/github/DoubekSeth/CS5350HW/blob/main/CS5350HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter

Creating Basic ID3 algorithm

In [ ]:
class Node:

  def __init__(self, s, atrs, parent=None, label=None):
    self.s = s
    self.atrs = atrs
    self.parent = parent
    self.label = label

In [ ]:
def ID3(s, remaining_attributes, full_attributes, info_gain_func, parentNode):
  #First, check if all training examples s have same label. First stopping condition
  if(labelSame(s)[0]):
    return Node(s, remaining_attributes, parent=parentNode, label=labelSame(s)[1])
  #Second, check if exhausted attributes and if so return leaf node with most common label. Second stopping condition
  if(len(remaining_attributes) == 0):
    return Node(s, remaining_attributes, parent=parentNode, label=mostCommonLabel(s, 1)[0][0])
  #If didn't stop, then we can move onto the main part of the algorithm
  #Find attribute best splits S
  bestAtr = findBestSplit(s, remaining_attributes, full_attributes, info_gain_func)

Does two functions, first returns true if all labels are the same for s. Second returns the label if all labels are same or else returns none

In [ ]:
def labelSame(s):
  label = s[0][len(s[0])-1]
  for i in s:
    if(i[len(i)-1] != label):
      return False, None
  return True, label

Another helper function, returns the n most common label in s

In [ ]:
def mostCommonLabel(s, n):
  labels = []
  for i in s:
    labels.append(i[len(i)-1])
  #Find most common label out of all the labels
  mostCommon = Counter(labels).most_common(n)
  return mostCommon

This helper finds the attribute that results in the best split

In [ ]:
def findBestSplit(s, atrs, full_attributes, info_gain_func):
  max_info_gain = 0
  max_info_gain_atr = ""
  for atr in atrs:
    info_gained = info_gain_func(s, atr, full_attributes)
    if(info_gained > max_info_gain):
      max_info_gain = info_gained
      max_info_gain_atr = atr
  return max_info_gain_atr

One of the info gain functions

In [ ]:
def entropy(s, atr, full_attributes):
  atr_ind = full_attributes.index(atr)
  atr_props = {}
  for i in s:
    atr_props[i[atr_ind]] =

SyntaxError: ignored

In [ ]:
entropy(s, "wind", attributes)

In [ ]:
mostCommonLabel(s, None)

In [ ]:
s = [["sunny", "hot", "high", "weak", "-"],
     ["sunny", "hot", "high", "strong", "-"],
     ["overcast", "hot", "high", "weak", "+"],
     ["rainy", "mild", "high", "weak", "+"],
     ["rainy", "cool", "normal", "weak", "+"],
     ["rainy", "cool", "normal", "strong", "-"],
     ["overcast", "cool", "normal", "strong", "+"],
     ["sunny", "mild", "high", "weak", "-"],
     ["sunny", "cool", "normal", "weak", "+"],
     ["rainy", "mild", "normal", "weak", "+"],
     ["sunny", "mild", "normal", "strong", "+"],
     ["overcast", "mild", "high", "strong", "+"],
     ["overcast", "hot", "normal", "weak", "+"],
     ["rainy", "mild", "high", "strong", "-"]]
attributes = ["outlook", "temperature", "humidity", "wind"]
info_gain_func = 2
rootNode = None
#ID3(s, attributes, info_gain_func, rootNode)